In [2]:
import numpy as np
import cv2 as cv
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt

In [3]:
frames= "/media/david/ssd1/dataset/kitti/odometry/dataset/depth_maps"
trajectories="/media/david/ssd1/dataset/kitti/odometry/dataset/trajectories/"
print (listdir(frames),len(listdir(frames)))

['03', '09', '06', '04', '02', '10', '00', '01', '05', '08', '07'] 11


In [4]:
def Euler2Quaternion(roll,pitch,yaw):
    cy = np.cos(yaw * 0.5);
    sy = np.sin(yaw * 0.5);
    cp = np.cos(pitch * 0.5);
    sp = np.sin(pitch * 0.5);
    cr = np.cos(roll * 0.5);
    sr = np.sin(roll * 0.5);

    w = cy * cp * cr + sy * sp * sr;
    x = cy * cp * sr - sy * sp * cr;
    y = sy * cp * sr + cy * sp * cr;
    z = sy * cp * cr - cy * sp * sr;

    return np.array([w,x,y,z])

def Hamiltonmult(quat1,quat2):
    return np.array([quat1[0]*quat2[0]-quat1[1]*quat2[1]-quat1[2]*quat2[2]-quat1[3]*quat2[3],
                     quat1[0]*quat2[1]+quat1[1]*quat2[0]+quat1[2]*quat2[3]-quat1[3]*quat2[2],
                     quat1[0]*quat2[2]-quat1[1]*quat2[3]+quat1[2]*quat2[0]+quat1[3]*quat2[1],
                     quat1[0]*quat2[3]+quat1[1]*quat2[2]-quat1[2]*quat2[1]+quat1[3]*quat2[0]])

In [5]:
X=listdir(frames)
X.sort()
C=1
C2=1
for route in X:
    directions=np.loadtxt(trajectories+route+".txt")
    translations=directions [:,1:4]
    rotations=directions [:,4:]
    Quat= np.array([Euler2Quaternion(phi,theta,psi) for phi,theta,psi in rotations])
    DltaT= translations[1:] - translations[:-1]
    ConQuat= Quat*[1,-1,-1,-1]
    DltaR=np.array([[0,0,0,0]])
    
    for i in range (1,len(Quat),1):
        dlta= np.array([Hamiltonmult(Quat[i],ConQuat[i-1])])
        DltaR=np.append(DltaR,dlta,axis=0)
    DltaR=np.delete(DltaR,0,axis=0)
    
    Dm_list=[f for f in listdir(frames+"/"+route) if f.startswith("depth_map_")]
    Dm_list=sorted(Dm_list,key=lambda name: int(name[10:-4]))
    
    Itsty_list=[f for f in listdir(frames+"/"+route) if f.startswith("intensity_map_")]
    Itsty_list=sorted(Itsty_list,key=lambda name: int(name[14:-4]))
    
    lbl_list=[f for f in listdir(frames+"/"+route) if f.startswith("label_map_")]
    lbl_list=sorted(lbl_list,key=lambda name: int(name[10:-4]))
    
    for de in range (0,len(lbl_list),1000):
        Tdmlist=Dm_list[de:de+1001]
        Titstylist=Itsty_list[de:de+1001]
        Tlbllist=lbl_list[de:de+1001]
        MatL= []
        D={}
        D["Traslations"]= DltaT[de:de+1000]
        D["Rotations"]= DltaR[de:de+1000]
        for i in range (len(Tdmlist)):
            Dm_image = cv.imread(frames+"/"+route+"/"+Tdmlist[i])
            Itsty_image = cv.imread(frames+"/"+route+"/"+Titstylist[i])
            lbl_image=cv.imread(frames+"/"+route+"/"+Tlbllist[i])
            mat=np.zeros(Dm_image.shape)
            mat[:,:,0]=Dm_image[:,:,0]/255.0
            mat[:,:,1]=Itsty_image[:,:,0]/255.0
            mat[:,:,2]=lbl_image[:,:,0]
            MatL.append(mat)
            #print(lbl_image.shape)
            #cv.namedWindow('ImageWindow1', cv.WINDOW_NORMAL)
            #cv.namedWindow('ImageWindow2', cv.WINDOW_NORMAL)
            #cv.namedWindow('ImageWindow3', cv.WINDOW_NORMAL)
            #cv.imshow('ImageWindow1', lbl_image[:,:,0] )
            #cv.imshow('ImageWindow2', lbl_image[:,:,1] )
            #cv.imshow('ImageWindow3', lbl_image[:,:,2] )
            #cv.waitKey(0)
            #cv.destroyAllWindows()
        D["depth_map"]=np.array(MatL)
        if (int(route)>=9):
            np.save("D_test_{}.npy".format(C2), D)
            C2+=1
        else:
            np.save("D_train_{}.npy".format(C), D)
            C+=1

In [61]:
d2=np.load("D_train_13.npy", allow_pickle=True)

In [62]:
print(d2[()]['Traslations'].shape)
print(d2[()]['Rotations'].shape)
print(d2[()]['depth_map'].shape)

(800, 3)
(800, 4)
(801, 32, 360, 3)
